In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import json
import os
import numpy as np
from utils import *
import math
import heapq

In [15]:
fd_collection = getCollection("team_5_mwdb_phase_2", "fd_collection")
all_images = fd_collection.find()

In [16]:
selected_latent_space = valid_latent_spaces[
    str(input("Enter latent space - one of " + str(list(valid_latent_spaces.keys()))))
]

selected_feature_model = valid_feature_models[
    str(input("Enter feature model - one of " + str(list(valid_feature_models.keys()))))
]

k = int(input("Enter value of k: "))
if k < 1:
    raise ValueError("k should be a positive integer")

if selected_latent_space != 'cp':
    selected_dim_reduction_method = str(
        input(
            "Enter dimensionality reduction method - one of "
            + str(list(valid_dim_reduction_methods.keys()))
        )
    )

label = int(input("Enter label: "))
if label < 0 and label > 100:
    raise ValueError("k should be between 0 and 100")

knum = int(input("Enter value of knum: "))
if knum < 1:
    raise ValueError("knum should be a positive integer")

label_rep = calculate_label_representatives(fd_collection, label, selected_feature_model)

match selected_latent_space:
    case "":
        if os.path.exists(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
            data = json.load(open(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
            print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json loaded")
        else:
          print(f"{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist")
    case "cp":
        if os.path.exists(f"{selected_feature_model}-cp-{k}-semantics.json"):
            data = json.load(open(f"{selected_feature_model}-cp-{k}-semantics.json"))
            print(f"{selected_feature_model}-cp-{k}-semantics.json loaded")
        else:          
          print(f"{selected_feature_model}-cp-{k}-semantics.json does not exist")
    case _:
        if os.path.exists(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"):
            data = json.load(open(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json"))
            print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json loaded")
        else:
          print(f"{selected_latent_space}-{selected_feature_model}-{selected_dim_reduction_method}-{k}-semantics.json does not exist")


cm_fd-cp-10-semantics.json loaded


In [17]:
def extract_similarities_ls1_ls4(latent_space, dim_reduction, data, label, label_rep):

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      if len(S.shape) == 1:
        S = np.diag(S)
      V = np.transpose(np.array(data["semantic-feature"]))

      if latent_space == "image_sim":
        label_vectors = []
        length = len(U)
        for i in range(length):
          if all_images[i]["true_label"] == label:
            label_vectors.append(U[i])
        label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]
      
      comparison_feature_space = np.matmul(U, S)

      if latent_space == "image_sim":
        print(np.array(label_rep).shape)
        print(np.array(S).shape)
        comparison_vector = np.matmul(label_rep, S)
      else:
        comparison_vector = np.matmul(np.matmul(label_rep, V), S)
    
    case "nmf":
      H = np.array(data['semantic-feature'])
      comparison_feature_space = W = np.array(data['image-semantic'])
      if latent_space == "image_sim":
        label_vectors = []
        length = len(W)
        for i in range(length):
          if all_images[i]["true_label"] == label:
            label_vectors.append(W[i])
        label_rep = [sum(col) / len(col) for col in zip(*label_vectors)]

      if latent_space == "image_sim":
        comparison_vector = label_rep
      else:
        comparison_vector = np.matmul(label_rep, np.transpose(H))

    case "kmeans":
      comparison_vector = []
      comparison_feature_space = np.array(data["image-semantic"])
      S = np.array(data["semantic-feature"])

      for centroid in S:
        comparison_vector.append(math.dist(label_rep, centroid))

  n = len(comparison_feature_space)

  distances = []
  for i in range(n):
      distances.append({"image_id": i, "label": all_images[i]["true_label"], "distance": math.dist(comparison_vector, comparison_feature_space[i])})

  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)

  similar_images = []

  for img in distances:
      similar_images.append(img)
      if len(similar_images) == knum:
        break


  for x in similar_images:
    print(x)

In [18]:
def extract_similarities_ls2(data, label):

  LS_f = np.array(data["feature-semantic"])
  LS_i = np.array(data["image-semantic"])
  S = np.array(data["semantics-core"])

  if len(S.shape) == 1:
    S = np.diag(S)

  label_rep = calculate_label_representatives(fd_collection, label, selected_feature_model)
  comparison_feature_space = np.matmul(label_rep, LS_f)
  comparison_vector = np.matmul(comparison_feature_space, S)

  comparison_image_space = np.matmul(LS_i, S)
  distances = []

  n = len(comparison_image_space)
  for i in range(n):
      distances.append({"image": i, "distance": math.dist(comparison_vector, comparison_image_space[i])})
  
  distances = sorted(distances, key=lambda x: x["distance"], reverse=False)[:knum]

  for x in distances:
    print(x)

In [19]:
def extract_similarities_ls3(dim_reduction, data, label):

  match dim_reduction:

    case 'svd':
      U = np.array(data["image-semantic"])
      S = np.array(data["semantics-core"])
      V = np.transpose(np.array(data["semantic-feature"]))

      comparison_feature_space = np.matmul(U, S)
      comparison_vector = comparison_feature_space[label]
    
    case "nmf":
      comparison_feature_space = np.array(data['image-semantic'])
      comparison_vector = comparison_feature_space[label]

    case "kmeans":
      comparison_feature_space = np.array(data["image-semantic"])
      comparison_vector = comparison_feature_space[label]

  n = len(comparison_feature_space)
  distances = []
  for i in range(n):
      if i != label:
        distances.append({"label": i, "distance": math.dist(comparison_vector, comparison_feature_space[i])})

  label_distance = sorted(distances, key=lambda x: x["distance"], reverse=False)[:1]

  dataset = loadDataset(Caltech101)
  similar_images = []
  for i in range(len(dataset)):
    _, l = dataset[i]
    if l == label:
      similar_images.append(i)

  similar_images = random.sample(similar_images, knum)
  images_distances = []
  for i in similar_images:
    images_distances.append({"image_id": i,"distance": label_distance[0]["distance"]})
    
  for x in images_distances:
    print(x)

In [20]:
match selected_latent_space:

  case "" | "image_sim":
    
    extract_similarities_ls1_ls4(selected_latent_space, selected_dim_reduction_method, data, label, label_rep)

  case "label_sim":

    extract_similarities_ls3(selected_dim_reduction_method, data, label)

  case "cp":

    extract_similarities_ls2(data, label)
    

{'image': 823, 'distance': 4006.335159603778}
{'image': 809, 'distance': 4006.3942621209867}
{'image': 806, 'distance': 4006.421689986329}
{'image': 832, 'distance': 4006.422683206996}
{'image': 830, 'distance': 4006.44733072835}
